## Solving bidomain equations on the ventricle with modified FitzHugh-Nagumo model

### Importing required libraries

In [ ]:
%cd ..
from src.models.dynamics_models import *
from src.models.cell_models import *
from src.utils import *

### Defining a domain, cell model and dynamics model and visualizing initial data

In [ ]:
domain = heart_ventricle(0.25)

In [ ]:
class Model(BidomainModel):
    def initial_V_m(self):
        def value(x):
            mu = -7
            alpha = 0.1
            return -85 + 35 / (1 + np.exp((x[2] - mu) / alpha))

        self.V_m_n.interpolate(value)
        return self.V_m_n

    def conductivity(self):
        # Muscle fibres
        x_c, y_c = 0.2, 1
        self.fibers = ufl.as_vector(
            [
                -(self.x[1] - y_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                (self.x[0] - x_c)
                / ufl.sqrt((self.x[0] - x_c) ** 2 + (self.x[1] - y_c) ** 2),
                0,
            ]
        )
        self.normals = ufl.as_vector([0, 0, 1])

        # Healthy conductivities
        self.M_i = (
            self.SIGMA_IT * ufl.Identity(3)
            + (self.SIGMA_IL - self.SIGMA_IT) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_IN - self.SIGMA_IT) * ufl.outer(self.normals, self.normals)
        )
        self.M_e = (
            self.SIGMA_ET * ufl.Identity(3)
            + (self.SIGMA_EL - self.SIGMA_ET) * ufl.outer(self.fibers, self.fibers)
            + (self.SIGMA_EN - self.SIGMA_ET) * ufl.outer(self.normals, self.normals)
        )

In [ ]:
cell_model = ReparametrizedFitzHughNagumo(domain)
model = Model(domain, cell_model)

In [ ]:
plot_function(
    model.V_m_n,
    save_to="ventricle_initial_V_m.pdf",
)

### Solving equations with given parameters

In [ ]:
model.solve(
    T=2000,
    steps=2000,
    save_to="ventricle.mp4",
    checkpoints=[500, 1000, 1500, 2000],
    checkpoint_file="ventricle",
)

### Plotting fiber orientations

In [ ]:
x_c, y_c = 0, 0
plot_vector_field(
    domain,
    lambda x: (
        -(x[1] - y_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        (x[0] - x_c) / np.sqrt((x[0] - x_c) ** 2 + (x[1] - y_c) ** 2),
        0,
    ),
    0.02,
    0.4,
    save_to="ventricle_fibers.pdf",
)